----------
# Preprocess 3 （Suburb Dataset）
1. Find Average Rental Price of Suburbs
2. Merge Population, Income
3. Merge Number of Schools, Hospitals and Train Stations
4. Generate Future Suburb Data.
------------

In [1]:
import numpy as np
import pandas as pd
import json
import seaborn as sns
import geopandas as gpd
from geopandas.tools import sjoin

/opt/anaconda3/envs/ads/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


-------------------
### Find Average Rental Price of Suburbs
load dataset

In [2]:
property = pd.read_csv("../data/curated/property_final.csv", index_col=[0])
property.head(2)

,type,cloest station,station_distance,cloest school,school_distance,CBD_distance,postcode,address,cost,beds,bath,parking,LOC_PID,LOC_NAME,suburb_population,density,offence_count_scaled,2022_income
0,Apartment / Unit / Flat,30,1803.65,1374,667.06,1702.19,3000,1901/368 St Kilda Road Melbourne,1800.0,3,2,2,loc9901d119afda,Melbourne,64537.612413,9902.586904,0.637907,59707.937643
1,Apartment / Unit / Flat,176,371.68,2265,352.49,2267.13,3000,1211/200 Spencer Street Melbourne,480.0,1,1,1,loc9901d119afda,Melbourne,64537.612413,9902.586904,0.637907,59707.937643


In [3]:
rental_price = property[['LOC_PID', 'LOC_NAME', 'cost']].groupby(["LOC_PID", "LOC_NAME"], as_index=False).mean()
rental_price.head()

,LOC_PID,LOC_NAME,cost
0,loc0067a4549ed1,Korumburra,278.333333
1,loc00a9769647d7,Kew,621.281250
2,loc00d1503504f1,Glen Waverley,561.269841
3,loc00e6e39d335b,Sailors Falls,450.000000
4,loc00f0949ea0ad,Sunbury,471.617647


-------------
### Merge Population, Income
Find population and income

In [4]:
pop_income = property[['LOC_PID', 'LOC_NAME', 'suburb_population', 'density', 'offence_count_scaled', '2022_income']].groupby(["LOC_PID", "LOC_NAME"]).apply(lambda x : x).drop_duplicates()
pop_income.head()

,LOC_PID,LOC_NAME,suburb_population,density,offence_count_scaled,2022_income
0,loc9901d119afda,Melbourne,64537.612413,9902.586904,0.637907,59707.937643
447,loc0b8afd71fce1,West Melbourne,10506.016393,1560.446386,0.258423,54498.763303
541,loc31f384e524fe,Southbank,26440.039991,16912.733621,0.285552,62372.938229
573,loc7024d58288dd,East Melbourne,5803.000000,3126.597105,0.396347,84739.514398
659,loc1e06c486c813,North Melbourne,15687.065574,6617.895792,0.223879,54498.763303


Merge

In [5]:
suburb_info = pd.merge(rental_price, pop_income.drop('LOC_NAME', axis=1), on='LOC_PID', how='left')
suburb_info = suburb_info.rename(columns={'cost' : 'rental_price', 'density' : 'population_density'})
suburb_info.head()

,LOC_PID,LOC_NAME,rental_price,suburb_population,population_density,offence_count_scaled,2022_income
0,loc0067a4549ed1,Korumburra,278.333333,4897.000000,81.797573,0.165407,47097.371969
1,loc00a9769647d7,Kew,621.281250,26158.000000,2486.279947,0.107577,71097.931084
2,loc00d1503504f1,Glen Waverley,561.269841,41928.000000,2489.998587,0.105967,44548.852850
3,loc00e6e39d335b,Sailors Falls,450.000000,1057.555556,108.121093,0.001891,46234.245242
4,loc00f0949ea0ad,Sunbury,471.617647,39266.294118,297.403254,0.130443,62622.919507


---------
### Merge number of Train Stations, Hospitals and Schools

Get Number of Train Stations

In [6]:
suburb_shp = gpd.read_file('../data/raw/external_data/vic_localities/vic_localities.shp')
suburb_shp = suburb_shp[['LOC_PID', 'LOC_NAME', 'geometry']]
train_station = gpd.read_file('../data/raw/train_station/ll_gda2020/esrishape/whole_of_dataset/victoria/TRANSPORT/VIC_RAILWAY_STATIONS.shp')
suburb_shp.head(2)

,LOC_PID,LOC_NAME,geometry
0,locb0dcb52a6b55,Abbeyard,"POLYGON ((146.81722 -37.09734, 146.81729 -37.0..."
1,locb9872f35df41,Abbotsford,"POLYGON ((145.00235 -37.80722, 145.00350 -37.8..."


In [7]:
train_station.head(2)

,STATION,STATUS,STOPID_MTR,STID_VLINE,PREMIUM,METRO,VLINE,INTERSTATE,STOP_ZONE,COMMENTS,geometry
0,Dandenong,Active,19888,22250,Yes,Yes,Yes,No,"Reg, 2",Metro Interchange,POINT (145.20964 -37.98992)
1,Darebin,Active,19932,None,No,Yes,No,No,1,None,POINT (145.03863 -37.77480)


In [8]:
suburb_train= sjoin(train_station[['STATION', 'geometry']], suburb_shp, how="left")  # Find Station suburb
num_stations = suburb_train[['STATION', 'LOC_PID', 'LOC_NAME']].groupby(['LOC_PID', 'LOC_NAME'], as_index=False).count()
num_stations = num_stations.rename(columns={'STATION' : 'num_stations'})
num_stations.head()

,LOC_PID,LOC_NAME,num_stations
0,loc00d1503504f1,Glen Waverley,2
1,loc00f0949ea0ad,Sunbury,1
2,loc01699d7adbaa,Birregurra,1
3,loc025dead673cc,Parkdale,1
4,loc05e50c5030cd,Ravenhall,1


Get Number of Hospitals

In [9]:
hospital = pd.read_csv("../data/raw/MainHospitalList.csv")
hospital.head()

,Hospital ID,Formal Name,Other Name,Emergency Capable,Location Address,Suburb,Postcode,Access Point,Category,Agency Type
0,5488,Albert Road Clinic,Albert Road Clinic,NO,31-33 Albert Road,South Melbourne,3205,NaN,PRIVATE,Private Hospital
1,3485,Albury Wodonga Health,Albury Wodonga Health,YES,69 Vermont Street,Wodonga,3690,NaN,PUBLIC,Public Hospital
2,12990,"Albury Wodonga Health, Albury Campus","Albury Wodonga Health, Albury Campus",YES,Borella,Albury,2640,NaN,PUBLIC,Public Hospital
3,3491,Alexandra District Health,Alexandra District Health,NO,20 Cooper Street,Alexandra,3714,NaN,PUBLIC,Public Hospital
4,11519,Alfred Health,Alfred Health,NO,Commercial Road,Melbourne,3004,NaN,PUBLIC,Public Hospital


In [10]:
num_hospitals = hospital[['Hospital ID', 'Suburb']].groupby('Suburb', as_index=False).count()
num_hospitals = num_hospitals.rename(columns={'Hospital ID' : 'num_hospitals'})
num_hospitals.head()

,Suburb,num_hospitals
0,Albury,1
1,Alexandra,1
2,Altona,1
3,Ararat,1
4,Ascot Vale,1


Get Number of Schools

In [11]:
schools = pd.read_csv("../data/raw/external_data/school_location.csv", encoding='cp1252')
schools = gpd.GeoDataFrame(
        schools, geometry=gpd.points_from_xy(schools.X, schools.Y))
schools = schools.set_crs(epsg=7844, inplace=True)
schools = schools[['SCHOOL_NO', 'geometry']]
schools.head()

,SCHOOL_NO,geometry
0,1,POINT (146.66660 -38.61771)
1,3,POINT (142.59039 -38.38628)
2,4,POINT (143.47565 -37.08450)
3,8,POINT (145.23472 -36.90137)
4,12,POINT (145.21398 -37.74267)


In [12]:
suburb_schools= sjoin(schools, suburb_shp, how="left")  # Find School suburb
num_schools = suburb_schools[['LOC_PID', 'LOC_NAME', 'SCHOOL_NO']].groupby(['LOC_PID', 'LOC_NAME'], as_index=False).count()
num_schools = num_schools.rename(columns={'SCHOOL_NO': 'num_schools'})
num_schools.head()

,LOC_PID,LOC_NAME,num_schools
0,loc00055278d626,Newstead,1
1,loc0035746c37e9,Coragulac,1
2,loc0067a4549ed1,Korumburra,3
3,loc00a571effff1,Nathalia,4
4,loc00a9769647d7,Kew,11


Merge number of hospitals, train stations and schools

In [13]:
curated_suburb = suburb_info
curated_suburb = pd.merge(curated_suburb, num_stations.drop('LOC_NAME', axis=1), on='LOC_PID', how='left')
curated_suburb = pd.merge(curated_suburb, num_schools.drop('LOC_NAME', axis=1), on='LOC_PID', how='left')
curated_suburb = pd.merge(curated_suburb, num_hospitals.rename(columns={'Suburb' : 'LOC_NAME'}), on='LOC_NAME', how='left')
curated_suburb = curated_suburb.rename(columns={'2022_income' : 'income'}).fillna(value=0)
curated_suburb

,LOC_PID,LOC_NAME,rental_price,suburb_population,population_density,offence_count_scaled,income,num_stations,num_schools,num_hospitals
0,loc0067a4549ed1,Korumburra,278.333333,4897.000000,81.797573,0.165407,47097.371969,0.0,3.0,0.0
1,loc00a9769647d7,Kew,621.281250,26158.000000,2486.279947,0.107577,71097.931084,0.0,11.0,6.0
2,loc00d1503504f1,Glen Waverley,561.269841,41928.000000,2489.998587,0.105967,44548.852850,2.0,13.0,2.0
3,loc00e6e39d335b,Sailors Falls,450.000000,1057.555556,108.121093,0.001891,46234.245242,0.0,0.0,0.0
4,loc00f0949ea0ad,Sunbury,471.617647,39266.294118,297.403254,0.130443,62622.919507,1.0,14.0,1.0
...,...,...,...,...,...,...,...,...,...,...
719,locff6258c8ea42,Montmorency,525.555556,9003.705882,2366.881699,0.050979,66315.679359,1.0,4.0,0.0
720,locff62fb6a898a,Carrum Downs,458.823529,22420.000000,1109.905643,0.165120,59654.130953,0.0,4.0,0.0
721,locffa1c8993b70,Mambourin,407.352941,5488.609375,154.062304,0.035164,59687.062945,0.0,0.0,0.0
722,locffb43e78ab10,Murtoa,300.000000,2133.333333,6.995021,0.056250,50199.145749,0.0,2.0,0.0


In [14]:
curated_suburb.to_csv('../data/curated/suburb_final.csv')

---------------
### Generate Future Suburb Data
Use predicted population income to generate future suburb data.

In [15]:
population_proj = pd.read_csv("../data/raw/population_projection.csv", index_col=[0])
population_proj = population_proj.drop('suburb_population', axis=1)
population_proj.head(2)

,LOC_PID,population_proj_2022,population_proj_2023,population_proj_2024,population_proj_2025
1,loc00a9769647d7,26760,27348,27923,28481
2,loc00d1503504f1,42892,43836,44757,45652


In [16]:
income = pd.read_csv('../data/raw/income_for_suburb.csv')
income.head(2)

,LOC_NAME,2011-12.3,2012-13.3,2013-14.3,2014-15.3,2015-16.3,2016-17.3,2017-18.3,2018-19.3,2019,2020,2021,2022,2023,2024,2025
0,Abbotsford,50034.0,50558.0,51419.0,53987.0,57501.0,58359.0,61476.0,64090.0,66409.949224,68813.876673,71304.822215,73885.935758,76560.481231,79331.840710,82203.518699
1,Aberfeldie,50458.0,52097.0,53037.0,54683.0,56381.0,56597.0,59116.0,61204.0,62920.085721,64684.288398,66497.957172,68362.479013,70279.279779,72249.825306,74275.622534


In [17]:
curated_suburb_2023 = curated_suburb.copy()
curated_suburb_2023['suburb_population'] = population_proj['population_proj_2023']
curated_suburb_2023 = pd.merge(curated_suburb_2023, income[['LOC_NAME', '2023']], on='LOC_NAME', how='left')
curated_suburb_2023 = curated_suburb_2023.drop('income', axis=1).rename(columns={'2023' : 'income'})
curated_suburb_2023.to_csv("../data/curated/suburb_2023.csv")

curated_suburb_2024 = curated_suburb.copy()
curated_suburb_2024['suburb_population'] = population_proj['population_proj_2024']
curated_suburb_2024 = pd.merge(curated_suburb_2024, income[['LOC_NAME', '2024']], on='LOC_NAME', how='left')
curated_suburb_2024 = curated_suburb_2024.drop('income', axis=1).rename(columns={'2024' : 'income'})
curated_suburb_2024.to_csv("../data/curated/suburb_2024.csv")

curated_suburb_2025 = curated_suburb.copy()
curated_suburb_2025['suburb_population'] = population_proj['population_proj_2025']
curated_suburb_2025 = pd.merge(curated_suburb_2025, income[['LOC_NAME', '2025']], on='LOC_NAME', how='left')
curated_suburb_2025 = curated_suburb_2025.drop('income', axis=1).rename(columns={'2025' : 'income'})
curated_suburb_2025.to_csv("../data/curated/suburb_2025.csv")